# Project 1: Population & Food Supply

In [41]:
# %pip install wbdata
# %pip install pandas
# %pip install plotly

import wbdata
import pandas as pd
import plotly.offline as py
import plotly.graph_objs as go
from datetime import datetime
import numpy as np

### Population Statistics Function

In [59]:
countries = wbdata.get_countries()
country_dict = {}

# Iterate over the WBSearchResult object, Create Dictionary.
# This dictionary is necessary so that we know which country
# names map to which 3-letter country codes.
for country in countries:
    country_code = country['id']
    country_name = country['name']

    # Add to the dictionary the country name.
    country_dict[country_name] = country_code

def int_to_str(num):
    """Convert the integer to the proper format."""
    if 0 <= num < 10:
        # Add a '0' prefix if it is a single digit.
        return f"0{num}"
    else:
        # Convert to string directly if it is a two-digit number.
        return str(num)

def population_range(year, sex, age_range, place):
    """This function will return the population for a certain age range."""
    sex_codes = {"people": "", "females": "FE", "males": "MA"}
    sex_used = sex_codes[sex]

    # Getting the lower and upper bounds for date into correct format.
    lower, upper = int_to_str(age_range[0]), int_to_str(age_range[1])
    range_string = lower + upper

    # Getting the country code via the name.
    country_code = country_dict.get(place)

    df = wbdata.get_dataframe({"SP.POP." + range_string + "." + sex_used: "Population"},
                              country= {country_code: place}).squeeze()
    df = df.to_frame().reset_index()
    population_total = int(df[df["date"] == str(year)]["Population"].iloc[0])
    return population_total

# function to return the population for people over 80
def over_80_pop(year, sex, place):
    sex_codes = {"people": "", "females": "FE", "males": "MA"}
    sex_used = sex_codes[sex]
    country_code = country_dict.get(place)

    df = wbdata.get_dataframe({"SP.POP." + "80UP" + "." + sex_used: "Population"},
                              country= {country_code: place}).squeeze()
    df = df.to_frame().reset_index()
    population_total = int(df[df["date"] == str(year)]["Population"].iloc[0])
    return population_total


def dict_helper(year, sex, age_range, place):
    """This will expand our function to include every age specified possible."""
    if len(age_range) == 1:
        age_range = [age_range[0], age_range[0]]
    elif age_range[1] < age_range[0]:
      raise ValueError(f"Please ensure that the second value in the range is greater than the first.")

    minimum_age, maximum_age = age_range
    possible_minimums = [i for i in range(0, 76, 5)]
    possible_maximums = [i for i in range(4, 80, 5)]

    my_dict = {}
    for age in range(minimum_age, maximum_age + 1):
        """Find the index in the possible ranges that includes the current age."""
        range_index = next((i for i, min_val in enumerate(possible_minimums) if
                            min_val <= age and age <= possible_maximums[i]), None)
        if range_index is not None:
            popl_value = population_range(year, sex, [possible_minimums[range_index], possible_maximums[range_index]], place) // 5
            my_dict[age] = popl_value
        elif age >= 80 and age <= 100:
            my_dict[age] = over_80_pop(year, sex, place) / 20
        else:
            raise ValueError(f"No age range available for age {age}")

    return my_dict


def population(year, sex, age_range, place):
    """This function ties everything together, returning population for given age ranges.
    During usage, please utilize the following format:
    Arguments to Use:
    Year (int): the specified year, works from 1960-2021.
    Sex (string): Anything from all, people, p, P, People, All, Everyone, female, females, f, Female, 
                Females, F, FE, male, males, m, Male, Males, M, MA works.
    Age Range (list with length 2, 2 integers): A list of the age bounds.
    Place (string): A string of the specified location.
    """
    if place not in country_dict:
        valid_regions = ", ".join(country_dict.keys())
        raise ValueError(f"The region '{place}' is not valid. Please choose from the following regions: {valid_regions}")
    if sex in ["all", "people", "p", "P", "People", "All", "Everyone"]:
      female_dict = dict_helper(year, "females", age_range, place)
      male_dict = dict_helper(year, "males", age_range, place)
      return sum(female_dict.values()) + sum(male_dict.values())
    elif sex in ["female", "females", "f", "Female", "Females", "F", "FE"]:
      female_dict = dict_helper(year, "females", age_range, place)
      return sum(female_dict.values())
    elif sex in ["male", "males", "m", "Male", "Males", "M", "MA"]:
      male_dict = dict_helper(year, "males", age_range, place)
      return sum(male_dict.values())
        
population(2023, "People", [5, 7], "Haiti")

736914

In [60]:
population(2022, "people", [0,100], "World")

7997756448.5

In [67]:
population(2023, "people", [0,100], "Haiti")

11640516.0

### Popualtion Dataframe

In [61]:
def create_population_dataframe(regions, years, age_range):
    """Creates the population DataFrame based on the regions, years, and age ranges wanted.
    During usage, please utilize the following format:
    Arguments to Use:
    regions (list of strings, any length): the specified year, works from 1960-2022.
    years (list of ints, any length): Anything from all, people, p, P, People, All, Everyone, female, females, f, Female, 
                Females, F, FE, male, males, m, Male, Males, M, MA works.
    age_range (list of two ints): the age bounds specified, second must be greater than the first.
    """
    data = []

    # Check if age_range is a single age or a range.
    if len(age_range) == 1:
        full_age_range = [age_range[0]]
    else:
        full_age_range = list(range(age_range[0], age_range[1] + 1))

    # Iterate over each region, year, and age."
    for region in regions:
        for year in years:
            row = {'Region': region, 'Year': year}
            for age in full_age_range:
                male_population = population(year, 'male', [age], region)
                female_population = population(year, 'female', [age], region)
                total_population = male_population + female_population

                row[f'Male Population Age {age}'] = male_population
                row[f'Female Population Age {age}'] = female_population
                row[f'Total Population Age {age}'] = total_population

            data.append(row)

    # Create a DataFrame.
    df = pd.DataFrame(data)

    # Set the index.
    df.set_index(['Region', 'Year'], inplace=True)

    return df

In [66]:
population_df = create_population_dataframe(["Haiti"], [1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2023], [0, 80])
population_df

Male Population Age 0  Female Population Age 0  \
Region Year                                                   
Haiti  1990                 110694                   109899   
       1991                 112761                   111911   
       1992                 114464                   113556   
       1993                 115758                   114788   
       1994                 116666                   115632   
       1995                 117289                   116189   
       1996                 117784                   116624   
       1997                 118296                   117073   
       1998                 118836                   117550   
       1999                 119400                   118068   
       2000                 120093                   118714   
       2001                 120750                   119312   
       2002                 121165                   119669   
       2003                 121539                   119966   
       2004                 121811                   120120   
       2005                 121932                   120105   
       2006                 122259                   120287   
       2007                 122914                   120782   
       2008                 123731                   121444   
       2009                 124681                   122269   
       2010                 124861                   122356   
       2011                 124937                   122388   
       2012                 125615                   123072   
       2023                 124824                   122286   

             Total Population Age 0  Male Population Age 1  \
Region Year                                                  
Haiti  1990                  220593                 110694   
       1991                  224672                 112761   
       1992                  228020                 114464   
       1993                  230546                 115758   
       1994                  232298                 116666   
       1995                  233478                 117289   
       1996                  234408                 117784   
       1997                  235369                 118296   
       1998                  236386                 118836   
       1999                  237468                 119400   
       2000                  238807                 120093   
       2001                  240062                 120750   
       2002                  240834                 121165   
       2003                  241505                 121539   
       2004                  241931                 121811   
       2005                  242037                 121932   
       2006                  242546                 122259   
       2007                  243696                 122914   
       2008                  245175                 123731   
       2009                  246950                 124681   
       2010                  247217                 124861   
       2011                  247325                 124937   
       2012                  248687                 125615   
       2023                  247110                 124824   

             Female Population Age 1  Total Population Age 1  \
Region Year                                                    
Haiti  1990                   109899                  220593   
       1991                   111911                  224672   
       1992                   113556                  228020   
       1993                   114788                  230546   
       1994                   115632                  232298   
       1995                   116189                  233478   
       1996                   116624                  234408   
       1997                   117073                  235369   
       1998                   117550                  236386   
       1999                   118068                  237